In [3]:
import duckdb
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os

input_csv = 'drug_docs_ph1to4_all.csv'
output_csv = 'drug_docs_with_refs - Copy.csv'

conn = duckdb.connect()

In [4]:
conn.execute(f"""
CREATE OR REPLACE TABLE papers AS 
SELECT * FROM read_csv_auto('{input_csv}')
""")

if os.path.exists(output_csv):
    # Load existing progress
    conn.execute(f"CREATE OR REPLACE TABLE papers_with_refs AS SELECT * FROM read_csv_auto('{output_csv}')")
    # Find how many rows already processed
    processed_count = conn.execute("SELECT COUNT(*) FROM papers_with_refs").fetchone()[0]
    print(f"Resuming from row {processed_count}")
else:
    # First run: create empty output table
    conn.execute("""
    CREATE TABLE papers_with_refs AS
    SELECT *, ''::VARCHAR AS references FROM papers WHERE 0
    """)
    processed_count = 0

# Fetch DOIs for next batch of up to 20,000 new rows
dois = conn.execute(f"""
SELECT doi FROM papers
ORDER BY doi
LIMIT 20000 OFFSET {processed_count}
""").fetchall()
dois = [d[0] for d in dois]

Resuming from row 200


In [5]:
def get_references(doi):
    if not doi:
        return ''
    try:
        response = requests.get(f'https://api.crossref.org/works/{doi}', timeout=10)
        if response.status_code == 200:
            data = response.json()
            refs = data.get('message', {}).get('reference', [])
            dois = [ref.get('DOI') for ref in refs if ref.get('DOI')]
            return ' '.join(dois)
    except:
        return ''
    return ''

def process_row(doi):
    return doi, get_references(doi)

In [ ]:
batch = []
batch_size = 100
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_row, doi) for doi in dois]
    for future in tqdm(as_completed(futures), total=len(futures)):
        doi, refs = future.result()
        batch.append((doi, refs))

        # Append batch to DuckDB table
        if len(batch) >= batch_size:
            conn.executemany("""
                INSERT INTO papers_with_refs (doi, references)
                VALUES (?, ?)
            """, batch)
            batch = []

# Insert any remaining rows
if batch:
    conn.executemany("""
        INSERT INTO papers_with_refs (doi, references)
        VALUES (?, ?)
    """, batch)

# Save DuckDB table to CSV
conn.execute(f"COPY papers_with_refs TO '{output_csv}' (HEADER, DELIMITER ',')")
print("Done")


  0%|          | 99/20000 [00:10<36:43,  9.03it/s]  


In [ ]:
import duckdb
import re

# Load patterns
with open('hit_synonyms.txt', 'r') as f:
    patterns = [line.strip() for line in f if line.strip()]

# Create regex pattern
combined_pattern = '|'.join(patterns)

# Load data into DuckDB
conn = duckdb.connect()
conn.execute("CREATE TABLE papers AS SELECT * FROM read_csv_auto('drug_docs_with_refs.csv')")

# Find citation pairs
query = f"""
SELECT DISTINCT 
    drg_paper.doi as drug_paper,
    drg_paper.title as drug_title,
    hit_paper.doi as hit_paper,
    hit_paper.title as hit_title
FROM papers drg_paper
JOIN papers hit_paper ON drg_paper.references LIKE '%' || hit_paper.doi || '%'
WHERE regexp_matches(hit_paper.title, '{combined_pattern}', 'i')
    AND drg_paper.year >= hit_paper.year
    AND drg_paper.doi != hit_paper.doi
"""

citation_pairs = conn.execute(query).df()
print(f'Found {len(citation_pairs)} citation pairs')
citation_pairs.head(10)

Found 0 citation pairs


,citing_paper,citing_title,cited_paper,cited_title
